In [4]:
#! pip install ibm_db
#! pip install pandas


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 45.1 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 46.6 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [5]:
import pandas as pd
import ibm_db_dbi

# Load CSV
# df = pd.read_csv("/Users/sakshimaurya/Desktop/Trace-finance/Version-2/extraction_from_bills_tradefinance.csv")
df = pd.read_csv("/Users/sakshimaurya/Desktop/Trade_final/db2_conn/clickstream_analytics.csv")


In [ ]:
df.head()

## DB2 connection


In [ ]:

try:
    conn_str = (
        "DATABASE=bludb;"
        "HOSTNAME="
        "PORT=;"
        "PROTOCOL=TCPIP;"
        "UID="
        "PWD=;"
        "SECURITY=SSL;"
    )
    conn = ibm_db.connect(conn_str, "", "")
    print("Connection successful.")
except Exception as e:
    print("Connection failed:")
    print(e)

# Scheme creation

In [ ]:
import ibm_db

schema_name = "TradeFinance"

try:
    create_schema_sql = f"CREATE SCHEMA {schema_name}"
    ibm_db.exec_immediate(conn, create_schema_sql)
    print(f"Schema '{schema_name}' created successfully.")
except Exception as e:
    print(f"Failed to create schema: {e}")

# Table creation

In [ ]:
table_name = "trade_data_2"

In [ ]:
columns_ddl = ", ".join([f'"{col}" VARCHAR(255)' for col in df.columns])
create_table_sql = f'CREATE TABLE "{schema_name}"."{table_name}" ({columns_ddl})'

try:
    ibm_db.exec_immediate(conn, create_table_sql)
    print(f"Table '{schema_name}.{table_name}' created.")
except Exception as e:
    print(f"Table creation skipped or failed: {e}")

# Insert data into table using ibm_db_dbi
conn_dbi = ibm_db_dbi.Connection(conn)

# Prepare insert statement
columns = [f'"{col}"' for col in df.columns]
placeholders = ", ".join(["?" for _ in df.columns])
insert_sql = f'INSERT INTO "{schema_name}"."{table_name}" ({", ".join(columns)}) VALUES ({placeholders})'


In [ ]:
# Insert data into table using ibm_db_dbi
conn_dbi = ibm_db_dbi.Connection(conn)

# Prepare insert statement
columns = [f'"{col}"' for col in df.columns]
placeholders = ", ".join(["?" for _ in df.columns])
insert_sql = f'INSERT INTO "{schema_name}"."{table_name}" ({", ".join(columns)}) VALUES ({placeholders})'

# Insert each row
try:
    cursor = conn_dbi.cursor()
    for row in df.itertuples(index=False, name=None):
        cursor.execute(insert_sql, row)
    cursor.close()
    print("All data inserted successfully.")
except Exception as e:
    print("Data insert failed:", e)

# Close connection
ibm_db.close(conn)